import models

In [145]:
import numpy as np
import cv2
import SimpleITK as sitk
from radiomics import featureextractor
import pandas as pd
import os

load data

In [ ]:
df_masstrain = pd.read_csv('preprocessed_data/fixed_mass_train_data_geo.csv')
df_masstest = pd.read_csv('preprocessed_data/fixed_mass_test_data_geo.csv')
df_calctrain = pd.read_csv('preprocessed_data/fixed_calc_train_data_geo.csv')
df_calctest = pd.read_csv('preprocessed_data/fixed_calc_test_data_geo.csv')

print column names

In [147]:
print(df_masstrain.columns)
print("\n")
print(df_masstest.columns)

Index(['patient_id', 'breast_density', 'left_or_right_breast', 'image_view',
       'abnormality_id', 'abnormality_type', 'mass_shape', 'mass_margins',
       'assessment', 'pathology', 'subtlety', 'image_file_path',
       'cropped_image_file_path', 'ROI_mask_file_path'],
      dtype='object')


Index(['patient_id', 'breast_density', 'left_or_right_breast', 'image_view',
       'abnormality_id', 'abnormality_type', 'mass_shape', 'mass_margins',
       'assessment', 'pathology', 'subtlety', 'image_file_path',
       'cropped_image_file_path', 'ROI_mask_file_path'],
      dtype='object')


In [ ]:
columns=[
    'center_x',
    'center_y',
    'roi_width',
    'roi_height'
]

df_masstrain.drop(columns=columns, inplace=True)
df_masstest.drop(columns=columns, inplace=True)
df_calctrain.drop(columns=columns, inplace=True)
df_calctest.drop(columns=columns, inplace=True)

In [148]:
extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.disableAllFeatures()

extractor.enableFeatureClassByName('firstorder')   # e.g., mean, std, entropy
extractor.enableFeatureClassByName('glcm')         # texture: contrast, homogeneity, etc.
# extractor.enableFeatureClassByName('glrlm')        # run-length encoding
# extractor.enableFeatureClassByName('glszm')        # size zone matrix
extractor.enableFeatureClassByName('shape2D')      # shape features for 2D masks
# extractor.enableFeatureClassByName('gldm')       # (optional) use only if needed
# extractor.enableFeatureClassByName('ngtdm')      # (optional) can be noisy, less interpretable

In [149]:
def pyradiomics_extractor(image_paths, mask_paths, extractor):

    features_list = []

    for img_path, mask_path in zip(image_paths, mask_paths):
        try:
            # Load image and mask
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            if img is None or mask is None:
                print(f"Could not read image or mask: {img_path} | {mask_path}")
                continue

            # Convert to SimpleITK
            sitk_image = sitk.Cast(sitk.GetImageFromArray(img), sitk.sitkFloat32)
            sitk_mask = sitk.Cast(sitk.GetImageFromArray(mask), sitk.sitkUInt8)

            # Ensure mask has labeled region
            if np.max(mask) == 0:
                print(f"No ROI in mask for: {img_path}")
                continue

            # Extract features
            result = extractor.execute(sitk_image, sitk_mask)
            features = {k: v for k, v in result.items() if k.startswith("original")}
            features['image_path'] = img_path
            features_list.append(features)

        except Exception as e:
            print(f"Error processing {img_path}: {e}")

    return features_list

extract features from mass dataset

In [150]:
image_paths = df_masstrain['image_file_path'].tolist()
mask_paths = df_masstrain['ROI_mask_file_path'].tolist()

radiomics_df_train = pd.DataFrame(pyradiomics_extractor(image_paths, mask_paths, extractor))

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.62520602613602980418030688082379323379/1-032.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 3133, 4686 ] which does not match the primary input's size of [ 3601, 5386 ]!
Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.43941248811932060425959307772414033323/1-131.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 3016, 4686 ] which does not match the primary input's size of [ 3466, 5386 ]!


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.20630952912877569514919614793206323155/1-164.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 2951, 4686 ] which does not match the primary input's size of [ 3391, 5386 ]!
Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.126154531211010377022690167651705249481/1-104.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 2990, 4686 ] which does not match the primary input's size of [ 3436, 5386 ]!


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

...

Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.223845628211173796734448562093137171383/1-263.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 2142, 4543 ] which does not match the primary input's size of [ 2461, 5221 ]!
Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.278571609413662223200958613891688785467/1-109.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 2363, 4556 ] which does not match the primary input's size of [ 2716, 5236 ]!


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.70055575411327114507601381440700639808/1-224.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 2468, 4373 ] which does not match the primary input's size of [ 2836, 5026 ]!


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.109260810611334542122453682220561164301/1-079.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 2872, 5548 ] which does not match the primary input's size of [ 3301, 6376 ]!
Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.65146854411116044628397515693673807312/1-236.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 2611, 5221 ] which does not match the primary input's size of [ 3001, 6001 ]!
Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.104604449710490862429464129700252625138/1-185.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkIm

In [151]:
image_paths = df_masstest['image_file_path'].tolist()
mask_paths = df_masstest['ROI_mask_file_path'].tolist()

radiomics_df_test = pd.DataFrame(pyradiomics_extractor(image_paths, mask_paths, extractor))

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.417157466613270187342040774840270966760/1-299.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 1789, 4386 ] which does not match the primary input's size of [ 2056, 5041 ]!
Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.259283265213236844915772927081591873003/1-122.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 2155, 4373 ] which does not match the primary input's size of [ 2476, 5026 ]!


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.317216124410630220004117216344047489299/1-182.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 2820, 4686 ] which does not match the primary input's size of [ 3241, 5386 ]!
Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.325293736011893421816901315930122472763/1-183.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 2781, 4621 ] which does not match the primary input's size of [ 3196, 5311 ]!
Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.272184723711385047630954095450427972031/1-184.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkI

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

...

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.140657909213236485612229551402733226044/1-122.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 3238, 4686 ] which does not match the primary input's size of [ 3721, 5386 ]!


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.176197299412099100034183343550119515253/1-132.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 2859, 4686 ] which does not match the primary input's size of [ 3286, 5386 ]!
Error processing jpeg/1.3.6.1.4.1.9590.100.1.2.116480947112670898133535663431673811690/1-133.jpg: Exception thrown in SimpleITK LabelStatisticsImageFilter_Execute: D:\a\SimpleITK\SimpleITK\Code\BasicFilters\src\sitkImageFilter.cxx:71:
sitk::ERROR: Input "labelImage" for "LabelStatisticsImageFilter" has size of [ 2690, 4686 ] which does not match the primary input's size of [ 3091, 5386 ]!


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

In [152]:
merged_df_train = df_masstrain.merge(radiomics_df_train, how='left', left_on='image_file_path', right_on='image_path')
merged_df_train.dropna(inplace=True)

In [153]:
merged_df_test = df_masstest.merge(radiomics_df_test, how='left', left_on='image_file_path', right_on='image_path')
merged_df_test.dropna(inplace=True)

In [154]:
merged_df_test.head()

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,mass_shape,mass_margins,assessment,pathology,...,original_glcm_InverseVariance,original_glcm_JointAverage,original_glcm_JointEnergy,original_glcm_JointEntropy,original_glcm_MCC,original_glcm_MaximumProbability,original_glcm_SumAverage,original_glcm_SumEntropy,original_glcm_SumSquares,image_path
0,P_00016,4,LEFT,CC,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,...,0.040346156118741275,2.6552426718544866,0.3743362740363728,1.7918546979228225,0.9999999999999006,0.5150603664385135,5.310485343708973,1.7515085418040819,1.016320766585639,jpeg/1.3.6.1.4.1.9590.100.1.2.2450631492112551...
1,P_00016,4,LEFT,MLO,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,...,0.07023371746935772,2.0027991180505214,0.3299230111747716,1.876228739916468,0.9536291980867647,0.45778417154739903,4.005598236101042,1.8059950224471109,0.5038430785008607,jpeg/1.3.6.1.4.1.9590.100.1.2.8595221461117050...
2,P_00017,2,LEFT,CC,1,mass,ROUND,CIRCUMSCRIBED,4,MALIGNANT,...,0.22947827693013478,1.6291250013343332,0.36186626417761436,1.6911567403033745,0.507889095662463,0.5143858628692659,3.2582500026686665,1.46167846337324,0.2331298856113474,jpeg/1.3.6.1.4.1.9590.100.1.2.2213118961289329...
3,P_00017,2,LEFT,MLO,1,mass,ROUND,ILL_DEFINED,4,MALIGNANT,...,0.09352854330708661,1.738918051910178,0.5300945068710052,1.2550941642509184,0.7573751776507487,0.6921537802566345,3.477836103820356,1.161565620943832,0.1926006191475689,jpeg/1.3.6.1.4.1.9590.100.1.2.2399490644120920...
4,P_00032,3,RIGHT,CC,1,mass,ROUND,OBSCURED,0,BENIGN,...,0.05796059640415653,3.2101784839974776,0.3421127579931116,1.8210651750007383,0.9534430425659787,0.4337968329825435,6.420356967994955,1.7631045785965827,0.4997618339748088,jpeg/1.3.6.1.4.1.9590.100.1.2.2150818187136005...


In [155]:
merged_df_train.to_csv(f"csv/pyradiomics_mass_train.csv", index=False)
merged_df_test.to_csv(f"csv/pyradiomics_mass_test.csv", index=False)

extract features from calc dataset